In [36]:
import pandas as pd

base_test = pd.read_csv('../frame_identification/data/fn1.7/test.csv')
test_1cf = base_test[base_test.apply(lambda x: len(x['frame_names'].split('~$~')) == 1, axis=1)]

test_1cf['lu_pos'] = test_1cf.apply(lambda x: x['lu_name'].split('.')[-1], axis=1)

In [19]:
# Randomly sample 3 negative samples for each sample in the test set
from Hydra.utils.data import get_framenet_metadata

def get_similar_samples(frame_lu_defs, frame_name, lu):
    lu_pos = lu.split('.')[-1]
    
    samples = frame_lu_defs.query(f'frame_name == "{frame_name}" and lu.str.endswith("{lu_pos}") and lu != "{lu}"')
    
    if len(samples) > 4:
        samples = samples.sample(4, random_state=0)
    
    if len(samples) < 4:
        extra_samples = frame_lu_defs.query(f'frame_name == "{frame_name}" and lu != "{lu}"')
        if len(extra_samples) > 4:
            extra_samples = extra_samples.sample(4, random_state=0)
        samples = pd.concat([samples, extra_samples]).drop_duplicates().iloc[:4]
        
    if len(samples) < 4:
        extra_samples = frame_lu_defs.query(f'lu.str.endswith("{lu_pos}") and lu != "{lu}"')
        if len(extra_samples) > 4:
            extra_samples = extra_samples.sample(4, random_state=0)
        samples = pd.concat([samples, extra_samples]).drop_duplicates().iloc[:4]
        
    if len(samples) < 4:
        extra_samples = frame_lu_defs.query(f'lu != "{lu}"')
        if len(extra_samples) > 4:
            extra_samples = extra_samples.sample(4, random_state=0)
        samples = pd.concat([samples, extra_samples]).drop_duplicates().iloc[:4]
    
    return samples

lu_manager, frame_info, frame_lu_defs = get_framenet_metadata()

test_uu = []

for i in range(len(test_1cf)):
    sample = test_1cf.iloc[i][['frame_names', 'lu_name', 'lu_pos']]
    similar_samples = get_similar_samples(frame_lu_defs, sample['frame_names'], sample['lu_name'])

    test_uu.append(similar_samples)
